# Notebook 3: Case Triage & Response Assistance

**User Story**: US3 — Case Triage & Response Assistance  
**Persona**: Customer Service Representative (CSR)  

Present a case to the Service Agent for priority recommendation, category,
draft response grounded in Knowledge Base articles, and source citations.
Write-back requires explicit CSR confirmation.

In [ ]:
# Cell 2: Environment + Auth Setup
import os
from pathlib import Path

from dotenv import load_dotenv

# Load environment variables
env_path = Path("../.env")
if env_path.exists():
    load_dotenv(env_path)

# Verify required environment variables
required_vars = [
    "AZURE_AI_PROJECT_ENDPOINT",
    "AZURE_OPENAI_DEPLOYMENT",
    "SF_INSTANCE_URL",
    "SF_ACCESS_TOKEN",
]
missing = [v for v in required_vars if not os.environ.get(v)]
if missing:
    raise OSError(f"Missing required environment variables: {missing}")

print("Environment configured successfully.")
print(f"  Azure AI endpoint: {os.environ['AZURE_AI_PROJECT_ENDPOINT'][:40]}...")
print(f"  Salesforce instance: {os.environ['SF_INSTANCE_URL']}")

In [ ]:
# Cell 3: Start MCP Servers + Connect with MCP Client
import asyncio
import subprocess
import sys
import time

import nest_asyncio
from mcp import ClientSession
from mcp.client.sse import sse_client

nest_asyncio.apply()

project_root = os.path.abspath("..")

# Start salesforce-crm MCP server (SSE)
CRM_PORT = 8102
crm_process = subprocess.Popen(
    [sys.executable, "-m", "mcp_servers.salesforce_crm.server"],
    env={**os.environ, "MCP_TRANSPORT": "sse", "FASTMCP_PORT": str(CRM_PORT)},
    cwd=project_root,
    stdout=subprocess.PIPE,
    stderr=subprocess.PIPE,
)

# Start salesforce-knowledge MCP server (SSE)
KB_PORT = 8103
kb_process = subprocess.Popen(
    [sys.executable, "-m", "mcp_servers.salesforce_knowledge.server"],
    env={**os.environ, "MCP_TRANSPORT": "sse", "FASTMCP_PORT": str(KB_PORT)},
    cwd=project_root,
    stdout=subprocess.PIPE,
    stderr=subprocess.PIPE,
)

time.sleep(3)

for name, proc in [("CRM", crm_process), ("Knowledge", kb_process)]:
    if proc.poll() is not None:
        stderr = proc.stderr.read().decode() if proc.stderr else ""
        raise RuntimeError(f"{name} MCP server failed to start: {stderr}")

print(f"✅ CRM MCP server started (PID: {crm_process.pid}) at http://127.0.0.1:{CRM_PORT}/sse")
print(f"✅ Knowledge MCP server started (PID: {kb_process.pid}) at http://127.0.0.1:{KB_PORT}/sse")


# Connect to both MCP servers
async def _connect(url):
    _sse_cm = sse_client(url=url)
    read_stream, write_stream = await _sse_cm.__aenter__()
    _sess_cm = ClientSession(read_stream, write_stream)
    session = await _sess_cm.__aenter__()
    await session.initialize()
    tools_result = await session.list_tools()
    return _sse_cm, _sess_cm, session, tools_result.tools

loop = asyncio.get_event_loop()

crm_sse_cm, crm_sess_cm, crm_session, crm_tools = loop.run_until_complete(
    _connect(f"http://127.0.0.1:{CRM_PORT}/sse")
)
kb_sse_cm, kb_sess_cm, kb_session, kb_tools = loop.run_until_complete(
    _connect(f"http://127.0.0.1:{KB_PORT}/sse")
)

# Build tool-name → session routing map
all_mcp_tools = list(crm_tools) + list(kb_tools)
tool_router: dict[str, ClientSession] = {}
for t in crm_tools:
    tool_router[t.name] = crm_session
for t in kb_tools:
    tool_router[t.name] = kb_session

print(f"✅ MCP sessions connected — {len(crm_tools)} CRM + {len(kb_tools)} KB tools")
for t in all_mcp_tools:
    print(f"   • {t.name}: {t.description[:70] if t.description else ''}")

In [ ]:
# Cell 4: Create OpenAI Client + System Prompt + Chat Helper
import asyncio
import json
from pathlib import Path as _Path

from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential

project_client = AIProjectClient(
    endpoint=os.environ["AZURE_AI_PROJECT_ENDPOINT"],
    credential=DefaultAzureCredential(),
)
client = project_client.get_openai_client()

system_prompt = _Path("../agents/service/system_prompt.md").read_text(encoding="utf-8")
MODEL = os.environ.get("AZURE_OPENAI_DEPLOYMENT", "gpt-4o")

# Convert MCP tools → OpenAI function-calling format
openai_tools = [
    {
        "type": "function",
        "function": {
            "name": t.name,
            "description": t.description or "",
            "parameters": t.inputSchema or {"type": "object", "properties": {}},
        },
    }
    for t in all_mcp_tools
]

async def _call_tool(name: str, arguments: dict) -> str:
    session = tool_router[name]
    result = await session.call_tool(name, arguments)
    return "\n".join(b.text for b in result.content if hasattr(b, "text")) or ""

def chat(user_message: str, *, history: list | None = None, max_rounds: int = 10) -> list:
    """Send a message, auto-execute MCP tool calls, return updated history."""
    if history is None:
        history = [{"role": "system", "content": system_prompt}]
    history.append({"role": "user", "content": user_message})
    loop = asyncio.get_event_loop()
    for _ in range(max_rounds):
        resp = client.chat.completions.create(model=MODEL, messages=history, tools=openai_tools, tool_choice="auto")
        msg = resp.choices[0].message
        history.append(msg)
        if not msg.tool_calls:
            break
        for tc in msg.tool_calls:
            args = json.loads(tc.function.arguments) if tc.function.arguments else {}
            tool_result = loop.run_until_complete(_call_tool(tc.function.name, args))
            history.append({"role": "tool", "tool_call_id": tc.id, "content": tool_result})
    return history

def show(history: list):
    """Display the last assistant message from history."""
    from IPython.display import Markdown, display
    for m in reversed(history):
        content = m.content if isinstance(m, dict) else m.content
        role = m.get("role") if isinstance(m, dict) else getattr(m, "role", None)
        if role == "assistant" and content:
            display(Markdown(content))
            return

print(f"✅ OpenAI client ready (model: {MODEL})")
print(f"✅ {len(openai_tools)} MCP tools mapped to OpenAI functions")

In [ ]:
# Cell 5: Triage request — "Triage case #12345 and suggest a response"

history = chat(
    "Triage case #12345 and suggest a response. "
    "Include priority recommendation, category, "
    "a draft response grounded in Knowledge Base articles, "
    "and cite the KB article sources."
)
show(history)

In [ ]:
# Cell 6: User confirms write-back — "Apply this triage"

history = chat(
    "Apply this triage. Update the case priority and category as recommended, "
    "and add the draft response as an internal comment.",
    history=history,
)
show(history)

In [ ]:
# Cell 7: Verify update in Salesforce

history = chat(
    "Show me the current details of case #12345 to confirm the update was applied.",
    history=history,
)
show(history)

In [ ]:
# Cell 8: Cleanup — close MCP sessions + stop servers
import asyncio


async def _cleanup():
    await crm_sess_cm.__aexit__(None, None, None)
    await crm_sse_cm.__aexit__(None, None, None)
    await kb_sess_cm.__aexit__(None, None, None)
    await kb_sse_cm.__aexit__(None, None, None)

asyncio.get_event_loop().run_until_complete(_cleanup())

crm_process.terminate()
crm_process.wait(timeout=5)
print(f"✅ CRM MCP server stopped (PID: {crm_process.pid})")

kb_process.terminate()
kb_process.wait(timeout=5)
print(f"✅ Knowledge MCP server stopped (PID: {kb_process.pid})")

print("\nCase triage session complete.")